# Finetune Llama-3 with LLaMA Factory

Please use a **free** Tesla T4 Colab GPU to run this!

Project homepage: https://github.com/hiyouga/LLaMA-Factory

## Install Dependencies

In [ ]:
%cd /content/
%rm -rf LLaMA-Factory
!git clone https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers==0.0.25
!pip install .[bitsandbytes]

/content
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 10515, done.
remote: Counting objects: 100% (1331/1331), done.
remote: Compressing objects: 100% (251/251), done.
remote: Total 10515 (delta 1157), reused 1168 (delta 1080), pack-reused 9184
Receiving objects: 100% (10515/10515), 214.36 MiB | 14.61 MiB/s, done.
Resolving deltas: 100% (7753/7753), done.
Updating files: 100% (211/211), done.
/content/LLaMA-Factory
assets/       docker-compose.yml  examples/  pyproject.toml  requirements.txt  src/
CITATION.cff  Dockerfile          LICENSE    README.md       scripts/          tests/
data/         evaluation/         Makefile   README_zh.md    setup.py
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-zneujmuq/unsloth_8a2b99d2572645e388b997b7403e2ae0
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-zneujmuq/unsloth_8a2b99d2572645e388b997b7403e2ae0
  Resolved https://github.com/unslothai/u

### Check GPU environment

In [ ]:
import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("Please set up a GPU before using LLaMA Factory: https://medium.com/mlearning-ai/training-yolov4-on-google-colab-316f8fff99c6")

## Update Identity Dataset

In [ ]:
import json

%cd /content/LLaMA-Factory/

NAME = "Llama-3"
AUTHOR = "LLaMA Factory"

with open("data/identity.json", "r", encoding="utf-8") as f:
  dataset = json.load(f)

# for sample in dataset:
#   sample["output"] = sample["output"].replace("NAME", NAME).replace("AUTHOR", AUTHOR)

# with open("data/identity.json", "w", encoding="utf-8") as f:
#   json.dump(dataset, f, indent=2, ensure_ascii=False)


/content/LLaMA-Factory


## Fine-tune model via LLaMA Board

In [ ]:
from llmtuner import create_ui

%cd /content/LLaMA-Factory/

create_ui().queue().launch(share=True,debug=True)

/content/LLaMA-Factory
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().

Could not create share link. Missing file: /usr/local/lib/python3.10/dist-packages/gradio/frpc_linux_amd64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_linux_amd64
2. Rename the downloaded file to: frpc_linux_amd64_v0.2
3. Move the file to this location: /usr/local/lib/python3.10/dist-packages/gradio


<IPython.core.display.Javascript object>

## Fine-tune model via Command Line

It takes ~30min for training.

In [ ]:
import os
from llmtuner import run_exp
from llmtuner.extras.misc import torch_gc

# Set the PYTORCH_CUDA_ALLOC_CONF environment variable
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Change the current directory to '/content/LLaMA-Factory/'
%cd /content/LLaMA-Factory/

# Define the output directory
output_dir = "/content/out_dir/out2"

# Check if the directory already exists
if not os.path.exists(output_dir):
    # If it doesn't exist, create it
    os.makedirs(output_dir)

data = {
    "stage": "sft",
    "do_train": True,
    "model_name_or_path": "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "dataset": "my_finetune,my_finetune",
    "template": "llama3",
    "finetuning_type": "lora",
    "lora_target": "all",
    "output_dir": output_dir,
    "per_device_train_batch_size": 1,  # Decrease batch size
    "gradient_accumulation_steps": 8,   # Increase accumulation steps
    "lr_scheduler_type": "cosine",
    "logging_steps": 10,
    "warmup_ratio": 0.1,
    "save_steps": 1000,
    "learning_rate": 5e-5,
    "num_train_epochs": 3.0,
    "max_samples": 250,  # Reduce max samples
    "max_grad_norm": 1.0,
    "quantization_bit": 4,
    "loraplus_lr_ratio": 16.0,
    "use_unsloth": True,
    "fp16": True,  # Use mixed precision training
}

run_exp(data)

torch_gc()


/content/LLaMA-Factory
05/01/2024 07:52:09 - WARNING - llmtuner.hparams.parser - We recommend enable `upcast_layernorm` in quantized training.


05/01/2024 07:52:09 - INFO - llmtuner.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16


INFO:llmtuner.hparams.parser:Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

...

05/01/2024 07:53:35 - INFO - llmtuner.train.utils - Using LoRA+ optimizer with loraplus lr ratio 16.00.


INFO:llmtuner.train.utils:Using LoRA+ optimizer with loraplus lr ratio 16.00.
[WARNING|logging.py:329] 2024-05-01 07:53:35,906 >> ==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 48 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 8
\        /    Total batch size = 8 | Total steps = 18
 "-____-"     Number of trainable parameters = 20,971,520


Step,Training Loss
10,0.775200


[INFO|<string>:474] 2024-05-01 07:55:58,177 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)


[INFO|trainer.py:3305] 2024-05-01 07:55:58,188 >> Saving model checkpoint to /content/out_dir/out2
[INFO|configuration_utils.py:726] 2024-05-01 07:55:58,418 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/efa44c86af4fcbbc3d75e6cb1c8bfaf7f5c7cfc1/config.json
[INFO|configuration_utils.py:789] 2024-05-01 07:55:58,420 >> Model config LlamaConfig {
  "_name_or_path": "meta-llama/Meta-Llama-3-8B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layer

***** train metrics *****
  epoch                    =        3.0
  total_flos               =   779180GF
  train_loss               =     0.4668
  train_runtime            = 0:02:22.27
  train_samples_per_second =      1.012
  train_steps_per_second   =      0.127


## Infer the fine-tuned model

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from llmtuner import ChatModel
from llmtuner.extras.misc import torch_gc

%cd /content/LLaMA-Factory/

chat_model = ChatModel(dict(
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit", # use bnb-4bit-quantized Llama-3-8B-Instruct model
  adapter_name_or_path="llama3_lora",            # load the saved LoRA adapters
  finetuning_type="lora",                  # same to the one in training
  template="llama3",                     # same to the one in training
  quantization_bit=4,                    # load 4-bit quantized model
  use_unsloth=True,                     # use UnslothAI's LoRA optimization for 2x faster generation
))

messages = []
while True:
  query = input("\nUser: ")
  if query.strip() == "exit":
    break

  if query.strip() == "clear":
    messages = []
    torch_gc()
    print("History has been removed.")
    continue

  messages.append({"role": "user", "content": query})     # add query to messages
  print("Assistant: ", end="", flush=True)
  response = ""
  for new_text in chat_model.stream_chat(messages):      # stream generation
    print(new_text, end="", flush=True)
    response += new_text
  print()
  messages.append({"role": "assistant", "content": response}) # add response to messages

torch_gc()

[INFO|tokenization_utils_base.py:2087] 2024-05-01 07:57:02,978 >> loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/efa44c86af4fcbbc3d75e6cb1c8bfaf7f5c7cfc1/tokenizer.json
[INFO|tokenization_utils_base.py:2087] 2024-05-01 07:57:02,979 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2087] 2024-05-01 07:57:02,983 >> loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/efa44c86af4fcbbc3d75e6cb1c8bfaf7f5c7cfc1/special_tokens_map.json
[INFO|tokenization_utils_base.py:2087] 2024-05-01 07:57:02,985 >> loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/efa44c86af4fcbbc3d75e6cb1c8bfaf7f5c7cfc1/tokenizer_config.json


/content/LLaMA-Factory


[WARNING|logging.py:314] 2024-05-01 07:57:03,428 >> Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


05/01/2024 07:57:03 - INFO - llmtuner.data.template - Replace eos token: <|eot_id|>


INFO:llmtuner.data.template:Replace eos token: <|eot_id|>
[INFO|configuration_utils.py:726] 2024-05-01 07:57:03,489 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/efa44c86af4fcbbc3d75e6cb1c8bfaf7f5c7cfc1/config.json
[INFO|configuration_utils.py:789] 2024-05-01 07:57:03,492 >> Model config LlamaConfig {
  "_name_or_path": "unsloth/llama-3-8b-Instruct-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "b

...

05/01/2024 07:57:03 - INFO - llmtuner.model.patcher - Using KV cache for faster generation.


INFO:llmtuner.model.patcher:Using KV cache for faster generation.


05/01/2024 07:57:03 - INFO - llmtuner.model.adapter - Fine-tuning method: LoRA


INFO:llmtuner.model.adapter:Fine-tuning method: LoRA


RuntimeError: Unsloth: `llama3_lora` is not a full model or a PEFT model.